# Green Fund Hack-a-thon
 

### Reagan Berhe

## Import dash, and its componet to run this code
* !pip install dash 
* !pip install dash-components
* !pip install dash-html

In [1]:
import pandas as pd 
import os
import plotly.express as px 
import plotly.graph_objects as go

import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_table
from dash.dependencies import Input, Output


# importing and clean data ( importing csv into pandas)

In [2]:

# Set your path to the folder containing the .csv files
PATH = '../data/Analysis/' # Use your path

# Fetch all files in path
fileNames = os.listdir(PATH)

# Filter file name list for files ending with .csv
fileNames = [file for file in fileNames if '_results.csv' in file]

# Loop over all files

my_list = []

for file in fileNames:
# Read .csv file and append to list
    df = pd.read_csv(PATH + file, index_col= None, header=0)
    df['Name'] = file.replace('_results.csv','')
    my_list.append(df)  


In [3]:
names = pd.read_excel('../data/Meter_Names_and_Labels.xslx')

In [4]:
# Concatenate my_list to a dataframe
energy_con_dataframe = pd.concat(my_list, axis=0, ignore_index=True)
energy_con_dataframe = energy_con_dataframe.round(1)
energy_con_dataframe.reset_index(inplace=True)

In [5]:
# Converting datatime to datetime type 
energy_con_dataframe['Datetime'] = pd.to_datetime(energy_con_dataframe['Datetime'])
energy_con_df = energy_con_dataframe

## Cleaning up data and creating lables for graphing

In [16]:

energy_con_df['year'] = energy_con_df.Datetime.dt.year
energy_con_df['month'] = energy_con_df.Datetime.dt.month_name()
energy_con_df['days'] = energy_con_df.Datetime.dt.day
energy_con_df['hours'] = energy_con_df.Datetime.dt.time

# energy_con_df= energy_con_df.drop(columns=['index'])
energy_con_df.head()

,Actual,Predicted,obs_ci_lower,obs_ci_upper,Datetime,Name,year,month,days,hours
0,11.0,14.7,9.9,19.5,2016-06-02 21:00:00+00:00,SRC_TX_2L1ABC,2016,June,2,21:00:00
1,3.0,13.3,8.5,18.1,2016-06-03 00:00:00+00:00,SRC_TX_2L1ABC,2016,June,3,00:00:00
2,3.0,13.3,8.5,18.1,2016-06-03 01:00:00+00:00,SRC_TX_2L1ABC,2016,June,3,01:00:00
3,3.0,13.2,8.4,18.0,2016-06-03 02:00:00+00:00,SRC_TX_2L1ABC,2016,June,3,02:00:00
4,2.0,13.1,8.4,17.9,2016-06-03 03:00:00+00:00,SRC_TX_2L1ABC,2016,June,3,03:00:00


In [17]:
energy_dataframe = energy_con_df.groupby(['Name','year','month','days','hours'])[['Actual','Predicted']].mean().round(2)
energy_dataframe.reset_index(inplace=True)
print(energy_dataframe)

                 Name  year      month  days     hours  Actual  Predicted
0            Baseball  2015      April     1  00:00:00    16.0       13.4
1            Baseball  2015      April     1  01:00:00     9.0        6.7
2            Baseball  2015      April     1  02:00:00    12.0        5.7
3            Baseball  2015      April     1  03:00:00    11.0        5.1
4            Baseball  2015      April     1  04:00:00    11.0        2.1
...               ...   ...        ...   ...       ...     ...        ...
3638721  WeilWinfield  2020  September    30  19:00:00    81.0      138.9
3638722  WeilWinfield  2020  September    30  20:00:00    73.0      140.1
3638723  WeilWinfield  2020  September    30  21:00:00    81.0      140.5
3638724  WeilWinfield  2020  September    30  22:00:00    81.0      140.2
3638725  WeilWinfield  2020  September    30  23:00:00    84.0      142.8

[3638726 rows x 7 columns]


# App Layout

<!-- # fig = px.bar(energy_dataframe, x='year', y="Actual", color="Name", barmode='group',
#                  title="Gender Earnings Disparity",
#                  labels={"salary":"Annual Salary (in thousands)"} # customize axis label
#                 )
 -->

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.express as px


app = dash.Dash(__name__)

df = energy_dataframe
available_indicators = df['Name'].unique()

app.layout = html.Div([
    html.Div([

        html.Div([
            dcc.Dropdown(
                id='crossfilter-xaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='Baseball'
            ),
            dcc.RadioItems(
                id='crossfilter-xaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ],
        style={'width': '49%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                id='crossfilter-yaxis-column',
                options=[{'label': i, 'value': i} for i in df['month']],
                value='1'
            ),
            dcc.RadioItems(
                id='crossfilter-yaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'})
    ], style={
        'borderBottom': 'thin lightgrey solid',
        'backgroundColor': 'rgb(250, 250, 250)',
        'padding': '10px 5px'
    }),

    html.Div([
        dcc.Graph(
            id='crossfilter-indicator-scatter'
        )
    ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
    html.Div([
        dcc.Graph(id='x-time-series'),
        dcc.Graph(id='y-time-series'),
    ], style={'display': 'inline-block', 'width': '49%'}),

    html.Div(dcc.Slider(
        id='crossfilter-year--slider',
        min=df['year'].min(),
        max=df['year'].max(),
        value=df['year'].max(),
        marks={str(year): str(year) for year in df['year'].unique()},
        step=None
    ), style={'width': '49%', 'padding': '0px 20px 20px 20px'})
])


@app.callback(
    dash.dependencies.Output('crossfilter-indicator-scatter', 'figure'),
    [dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-xaxis-type', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-type', 'value'),
     dash.dependencies.Input('crossfilter-year--slider', 'value')])
def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type,
                 year_value):
    dff = df[df['year'] == year_value]

    fig = px.scatter(x=dff[dff['Name'] == xaxis_column_name]['Actual'],
            y=dff[dff['Name'] == yaxis_column_name]['Predicted'],
            hover_name=dff[dff['Name'] == yaxis_column_name]['Name']
            )

    fig.update_traces(customdata=dff[dff['Name'] == yaxis_column_name]['Name'])

    fig.update_xaxes(title=xaxis_column_name, type='linear' if xaxis_type == 'Linear' else 'log')

    fig.update_yaxes(title=yaxis_column_name, type='linear' if yaxis_type == 'Linear' else 'log')

    fig.update_layout(margin={'l': 40, 'b': 40, 't': 10, 'r': 0}, hovermode='closest')

    return fig


def create_time_series(dff, axis_type, title):

    fig = px.scatter(dff, x='year', y='Actual')

    fig.update_traces(mode='lines+markers')

    fig.update_xaxes(showgrid=False)

    fig.update_yaxes(type='linear' if axis_type == 'Linear' else 'log')

    fig.add_annotation(x=0, y=0.85, xanchor='left', yanchor='bottom',
                       xref='paper', yref='paper', showarrow=False, align='left',
                       bgcolor='rgba(255, 255, 255, 0.5)', text=title)

    fig.update_layout(height=225, margin={'l': 20, 'b': 30, 'r': 10, 't': 10})

    return fig


@app.callback(
    dash.dependencies.Output('x-time-series', 'figure'),
    [dash.dependencies.Input('crossfilter-indicator-scatter', 'hoverData'),
     dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-xaxis-type', 'value')])
def update_y_timeseries(hoverData, xaxis_column_name, axis_type):
    country_name = hoverData['points'][0]['customdata']
    dff = df[df['Name'] == country_name]
    dff = dff[dff['Name'] == xaxis_column_name]
    title = '<b>{}</b><br>{}'.format(country_name, xaxis_column_name)
    return create_time_series(dff, axis_type, title)


@app.callback(
    dash.dependencies.Output('y-time-series', 'figure'),
    [dash.dependencies.Input('crossfilter-indicator-scatter', 'hoverData'),
     dash.dependencies.Input('crossfilter-yaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-type', 'value')])
def update_x_timeseries(hoverData, yaxis_column_name, axis_type):
    dff = df[df['Name'] == hoverData['points'][0]['customdata']]
    dff = dff[dff['Name'] == yaxis_column_name]
    return create_time_series(dff, axis_type, yaxis_column_name)


if __name__ == '__main__':
    app.run_server(debug=False)

In [ ]:
import plotly.graph_objects as go
import numpy as np
np.random.seed(42)

# Simulate data
returns = np.random.normal(0.01, 0.2, 100)
price = 100 * np.exp(returns.cumsum())
time = np.arange(100)

layout = go.Layout(
    title="Historic Prices",
    plot_bgcolor="#FFF",  # Sets background color to white
    xaxis=dict(
        title="time",
        linecolor="#BCCCDC",  # Sets color of X-axis line
        showgrid=False  # Removes X-axis grid lines
    ),
    yaxis=dict(
        title="price",  
        linecolor="#BCCCDC",  # Sets color of Y-axis line
        showgrid=False,  # Removes Y-axis grid lines    
    )
)

fig = go.Figure(
    data=go.Scatter(x=time, y=price),
    layout=layout
)   
fig.show()

In [ ]:
import plotly.graph_objects as go
import numpy as np
np.random.seed(42)

# Simulate data
returns_A = np.random.normal(0.01, 0.2, 100)
returns_B = np.random.normal(0.01, 0.2, 100)
returns = np.append(returns_A, returns_B)

prices_A = 100 * np.exp(returns_A.cumsum())
prices_B = 100 * np.exp(returns_B.cumsum())
prices = np.append(prices_A, prices_B)

companies = ["A"] * 100 + ["B"] * 100
time = np.append(np.arange(100), np.arange(100))

df = pd.DataFrame({
    "company": companies,
    "time": time,
    "price": prices,
    "returns": returns
})

# Build graph
COLORS_MAPPER = {
    "A": "#38BEC9",
    "B": "#D64545"
}

layout = go.Layout(
    title="Performance of A vs. B",    
    plot_bgcolor="#FFFFFF",
    barmode="stack",
    xaxis=dict(
        domain=[0, 0.5],
        title="time",
        linecolor="#BCCCDC",
    ),
    yaxis=dict(
        title="price",
        linecolor="#BCCCDC"
    ),
    xaxis2=dict(
        domain=[0.6, 1],
        title="returns",
        linecolor="#BCCCDC",
    ),
    yaxis2=dict(
        anchor="x2",
        linecolor="#BCCCDC"
    )
)

data = []
for company,col in COLORS_MAPPER.items():
    time = df.loc[df.company == company, "time"]
    price = df.loc[df.company == company, "price"]
    returns = df.loc[df.company == company, "returns"]
    line_chart = go.Scatter(
        x=time,
        y=price,
        marker_color=col,  # Defines specific color for a trace
        legendgroup=company,  # Groups traces belonging to the same group in the legend 
        name=company
    )
    histogram = go.Histogram(
        x=returns,
        marker_color=col,  # Defines specific color for a trace
        legendgroup=company,  # Groups traces belonging to the same group in the legend 
        xaxis="x2", 
        yaxis="y2",
        showlegend=False
    )
    data.append(line_chart)
    data.append(histogram)

fig = go.Figure(data=data, layout=layout)
fig.show()

In [ ]:
import plotly.graph_objects as go
import numpy as np
np.random.seed(42)

# Simulate data
returns_A = np.random.normal(0.01, 0.2, 100)
returns_B = np.random.normal(0.01, 0.2, 100)
returns = np.append(returns_A, returns_B)

prices_A = 100 * np.exp(returns_A.cumsum())
prices_B = 100 * np.exp(returns_B.cumsum())
prices = np.append(prices_A, prices_B)

companies = ["A"] * 100 + ["B"] * 100
time = np.append(np.arange(100), np.arange(100))

df = pd.DataFrame({
    "company": companies,
    "time": time,
    "price": prices,
    "returns": returns
})

# Build graph
layout = go.Layout(
    title="Performance of A vs. B",    
    plot_bgcolor="#FFFFFF",
    hovermode="x",
    hoverdistance=100, # Distance to show hover label of data point
    spikedistance=1000, # Distance to show spike
    xaxis=dict(
        title="time",
        linecolor="#BCCCDC",
        showspikes=True, # Show spike line for X-axis
        # Format spike
        spikethickness=2,
        spikedash="dot",
        spikecolor="#999999",
        spikemode="across",
    ),
    yaxis=dict(
        title="price",
        linecolor="#BCCCDC"
    )
)

data = []
for company in ["A", "B"]:
    time = df.loc[df.company == company, "time"]
    price = df.loc[df.company == company, "price"]
    returns = df.loc[df.company == company, "returns"]
    line_chart = go.Scatter(
        x=time,
        y=price,
        name=company
    )
    data.append(line_chart)

fig = go.Figure(data=data, layout=layout)
fig.show()